# Demo - Performance Comparison (CIFAR10)

## 1. Load CIAFR10

In [1]:
# https://github.com/RobustBench/robustbench
from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from utils import l2_distance

images, labels = load_cifar10(n_examples=10)
device = "cuda"

Files already downloaded and verified


## 2. Standard Accuracy

In [2]:
model_list = ['Standard', 'Wong2020Fast', 'Rice2020Overfitting']
for model_name in model_list:
    model = load_model(model_name, norm='Linf').to(device)
    acc = clean_accuracy(model, images.to(device), labels.to(device))
    print('Model: {}'.format(model_name))
    print('- Standard Acc: {}'.format(acc))

Model: Standard
- Standard Acc: 1.0
Model: Wong2020Fast
- Standard Acc: 1.0
Model: Rice2020Overfitting
- Standard Acc: 0.8


## 3. Comparison with Foolbox and ART

In [3]:
import datetime
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

import torch
import torch.nn as nn
import torch.optim as optim

# https://github.com/bethgelab/foolbox
import foolbox as fb
print("foolbox %s"%(fb.__version__))

# https://github.com/IBM/adversarial-robustness-toolbox
import art
import art.attacks.evasion as evasion
from art.classifiers import PyTorchClassifier
print("art %s"%(art.__version__))

import sys
sys.path.insert(0, '..')
# https://github.com/Harry24k/adversarial-attacks-pytorch
import torchattacks
print("torchattacks %s"%(torchattacks.__version__))

foolbox 3.3.1
art 1.7.0
torchattacks 3.5.0


<ipython-input-3-363377dfba62>:18: DeprecationWarning: The module art.classifiers will be removed in ART 1.8.0 and replaced with art.estimators.classification
  from art.classifiers import PyTorchClassifier


## 3.1. Linf

### FGSM

In [4]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.FGSM(model, eps=8/255)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfFastGradientAttack(random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.FastGradientMethod(norm=np.inf, batch_size=50,
                                     estimator=classifier, eps=8/255)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

C:\Users\slcf\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\slcf\anaconda3\lib\site-packages\robustbench\utils.py:109: DeprecationWarning: `norm` has been deprecated and will be removed in a future version.
  warnings.warn(


Model: Standard
- Torchattacks
- Robust Acc: 0.4 (21 ms)
- Foolbox
- Robust Acc: 0.4 (10 ms)
- ART


C:\Users\slcf\anaconda3\lib\site-packages\art\attacks\evasion\fast_gradient.py:362: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if not isinstance(self.num_random_init, (int, np.int)):
C:\Users\slcf\anaconda3\lib\site-packages\art\attacks\evasion\fast_gradient.py:502: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-n

- Robust Acc: 0.4 (80 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.6 (7 ms)
- Foolbox
- Robust Acc: 0.6 (6 ms)
- ART
- Robust Acc: 0.6 (19 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.6 (21 ms)
- Foolbox
- Robust Acc: 0.6 (37 ms)
- ART
- Robust Acc: 0.6 (269 ms)



### BIM

In [5]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=10)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfBasicIterativeAttack(abs_stepsize=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.BasicIterativeMethod(batch_size=50,
                                       estimator=classifier, eps=8/255,
                                       eps_step=2/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Standard
- Torchattacks
- Robust Acc: 0.0 (101 ms)
- Foolbox
- Robust Acc: 0.0 (116 ms)
- ART


C:\Users\slcf\anaconda3\lib\site-packages\art\attacks\evasion\projected_gradient_descent\projected_gradient_descent.py:235: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if not isinstance(self.num_random_init, (int, np.int)):
C:\Users\slcf\anaconda3\lib\site-packages\art\attacks\evasion\projected_gradient_descent\projected_gradient_descent_numpy.py:207: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.0 (455 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.4 (65 ms)
- Foolbox
- Robust Acc: 0.4 (65 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.4 (136 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.6 (486 ms)
- Foolbox
- Robust Acc: 0.6 (651 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.6 (1747 ms)



### PGD

In [6]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfPGD(abs_stepsize=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.ProjectedGradientDescent(batch_size=50, num_random_init=0,
                                           estimator=classifier, eps=8/255,
                                           eps_step=2/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Standard
- Torchattacks
- Robust Acc: 0.0 (59 ms)
- Foolbox
- Robust Acc: 0.0 (111 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.0 (447 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.4 (61 ms)
- Foolbox
- Robust Acc: 0.4 (64 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.4 (146 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.6 (490 ms)
- Foolbox
- Robust Acc: 0.6 (656 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.6 (1713 ms)



## 3.2. L2

## DeepFool

In [7]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))

    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.L2DeepFoolAttack(steps=50, candidates=10, overshoot=0.02)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda'), labels.to('cuda'), epsilons=1)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))

    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.DeepFool(classifier=classifier, max_iter=50,
                           batch_size=50)

    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

Model: Standard
- Torchattacks
- Robust Acc: 0.0 / L2: 0.16 (2609 ms)
- Foolbox
- Robust Acc: 0.0 / L2: 0.16 (1847 ms)
- ART


C:\Users\slcf\anaconda3\lib\site-packages\art\attacks\evasion\deepfool.py:221: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if not isinstance(self.max_iter, (int, np.int)) or self.max_iter <= 0:
C:\Users\slcf\anaconda3\lib\site-packages\art\attacks\evasion\deepfool.py:224: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If 

DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.2 / L2: 0.19 (14928 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.0 / L2: 0.79 (695 ms)
- Foolbox
- Robust Acc: 0.4 / L2: 0.46 (306 ms)
- ART


DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.0 / L2: 0.78 (333 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.0 / L2: 0.61 (5979 ms)
- Foolbox
- Robust Acc: 0.1 / L2: 0.53 (5625 ms)
- ART


DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.0 / L2: 0.66 (5460 ms)



### CW

In [8]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.CW(model, c=1, kappa=0, steps=100, lr=0.01)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.L2CarliniWagnerAttack(binary_search_steps=1, initial_const=1,
                                           confidence=0, steps=100, stepsize=0.01)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda'), labels.to('cuda'), epsilons=1)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))

    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.CarliniL2Method(batch_size=50, classifier=classifier, 
                                  binary_search_steps=1, initial_const=1,
                                  confidence=0, max_iter=100,
                                  learning_rate=0.01)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

Model: Standard
- Torchattacks
- Robust Acc: 0.0 / L2: 0.34 (814 ms)
- Foolbox
- Robust Acc: 0.0 / L2: 0.31 (844 ms)
- ART


C:\Users\slcf\anaconda3\lib\site-packages\art\attacks\evasion\carlini.py:491: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if not isinstance(self.binary_search_steps, (int, np.int)) or self.binary_search_steps < 0:
C:\Users\slcf\anaconda3\lib\site-packages\art\attacks\evasion\carlini.py:494: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify

C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.0 / L2: 0.41 (37716 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.0 / L2: 0.57 (995 ms)
- Foolbox
- Robust Acc: 0.1 / L2: 0.5 (1138 ms)
- ART


C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.2 / L2: 0.71 (11814 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.2 / L2: 0.49 (14946 ms)
- Foolbox
- Robust Acc: 0.2 / L2: 0.49 (15185 ms)
- ART


C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.2 / L2: 0.57 (132464 ms)



### PGD L2

In [9]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').cuda()
    
    print("- Torchattacks")
    atk = torchattacks.PGDL2(model, eps=128/255, alpha=15/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.L2PGD(abs_stepsize=15/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=128/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.ProjectedGradientDescent(batch_size=50, num_random_init=0,
                                           norm = 2, estimator=classifier, eps=128/255,
                                           eps_step=15/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

Model: Standard
- Torchattacks
- Robust Acc: 0.0 / L2: 0.42 (60 ms)
- Foolbox
- Robust Acc: 0.0 / L2: 0.42 (120 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.0 / L2: 0.42 (454 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.7 / L2: 0.5 (45 ms)
- Foolbox
- Robust Acc: 0.7 / L2: 0.5 (57 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.7 / L2: 0.5 (139 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.7 / L2: 0.5 (463 ms)
- Foolbox
- Robust Acc: 0.7 / L2: 0.5 (670 ms)
- ART


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

- Robust Acc: 0.7 / L2: 0.5 (1673 ms)



## 4. Comparison with AutoAttack

In [4]:
# https://github.com/fra31/auto-attack
from autoattack import AutoAttack

C:\Users\slcf\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.AutoAttack(model, eps=8/255)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Torchattacks")
    atk = AutoAttack(model, norm='Linf', eps=8/255, version='standard')
    start = datetime.datetime.now()
    adv_images = atk.run_standard_evaluation(images, labels, bs=len(images)).cuda()
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Standard
- Torchattacks
- Robust Acc: 0.0 (497 ms)
- Torchattacks
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square
initial accuracy: 100.00%
apgd-ce - 1/1 - 10 out of 10 successfully perturbed
robust accuracy after APGD-CE: 0.00% (total time 3.9 s)
max Linf perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 0.00%
- Robust Acc: 0.0 (3888 ms)

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.3 (24015 ms)
- Torchattacks
setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square
initial accuracy: 100.00%
apgd-ce - 1/1 - 6 out of 10 successfully perturbed
robust accuracy after APGD-CE: 40.00% (total time 1.0 s)
apgd-t - 1/1 - 1 out of 4 successfully perturbed
robust accuracy after APGD-T: 30.00% (total time 8.7 s)
fab-t - 1/1 - 0 out of 3 successfully perturbed
robust accuracy after FAB-T: 30.00% (total time 23.3 s)
square - 1/1 - 0 out of 3 successf